# 1. Mounting virtual drive for Google Colab

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Importing libraries and datasets

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, auc, roc_curve, precision_score, f1_score, roc_auc_score, precision_recall_curve
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier

In [4]:
df_claims = pd.read_csv("/content/drive/MyDrive/claim_lines.csv")
df_claims.head(10)

,record_id,member_id,date_svc,diag1
0,57738,M0000001,2015-12-06,N92.6
1,57750,M0000001,2015-12-06,O26.842
2,65072,M0000001,2015-12-13,O26.842
3,201796,M0000001,2016-02-29,O26.843
4,267197,M0000001,2016-03-27,O26.843
5,284553,M0000001,2016-04-03,O26.843
6,301751,M0000001,2016-04-10,O26.843
7,304120,M0000001,2016-04-11,O26.843
8,309100,M0000001,2016-04-13,O34.21
9,309112,M0000001,2016-04-13,O80


In [5]:
df_claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1919983 entries, 0 to 1919982
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   record_id  int64 
 1   member_id  object
 2   date_svc   object
 3   diag1      object
dtypes: int64(1), object(3)
memory usage: 58.6+ MB


In [6]:
df_ccs = pd.read_csv("/content/drive/MyDrive/ccs.csv")
df_ccs.head(10)

,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol...",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection
2,A009,"Cholera, unspecified",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection
3,A0100,"Typhoid fever, unspecified",Diseases of the digestive system,Intestinal infection [135.],Intestinal infection
4,A0101,Typhoid meningitis,Diseases of the nervous system and sense organs,Central nervous system infection,Meningitis (except that caused by tuberculosis...
5,A0102,Typhoid fever with heart involvement,Infectious and parasitic diseases,Bacterial infection,Bacterial infection; unspecified site
6,A0103,Typhoid pneumonia,Diseases of the respiratory system,Respiratory infections,Pneumonia (except that caused by tuberculosis ...
7,A0104,Typhoid arthritis,Diseases of the musculoskeletal system and con...,Infective arthritis and osteomyelitis (except ...,Infective arthritis and osteomyelitis (except ...
8,A0105,Typhoid osteomyelitis,Diseases of the musculoskeletal system and con...,Infective arthritis and osteomyelitis (except ...,Infective arthritis and osteomyelitis (except ...
9,A0109,Typhoid fever with other complications,Diseases of the digestive system,Intestinal infection [135.],Intestinal infection


In [7]:
df_ccs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72167 entries, 0 to 72166
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   diag        72167 non-null  object
 1   diag_desc   72167 non-null  object
 2   ccs_1_desc  72167 non-null  object
 3   ccs_2_desc  72167 non-null  object
 4   ccs_3_desc  72167 non-null  object
dtypes: object(5)
memory usage: 2.8+ MB


In [8]:
df_drugs = pd.read_csv("/content/drive/MyDrive/prescription_drugs.csv")
df_drugs.head()

,record_id,member_id,date_svc,ndc,drug_category,drug_group,drug_class
0,4115084976453758912,M0023556,2016-05-08,51285040702,Estrogens,Estrogens,Estrogens
1,1750642805638674193,M0087538,2016-12-05,50474080303,Antiparkinson and Related Therapy Agents,Antiparkinson Dopaminergics,Nonergoline Dopamine Receptor Agonists
2,5543689263541245391,M0049608,2018-01-18,3089421,Anticoagulants,Direct Factor Xa Inhibitors,Direct Factor Xa Inhibitors
3,5952194046467620061,M0175153,2017-01-28,603580321,Gastrointestinal Agents - Misc.,Inflammatory Bowel Agents,Inflammatory Bowel Agents
4,1809570950798791089,M0152187,2016-05-13,591396501,Progestins,Progestins,Progestins


In [9]:
df_drugs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005934 entries, 0 to 3005933
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   record_id      int64 
 1   member_id      object
 2   date_svc       object
 3   ndc            int64 
 4   drug_category  object
 5   drug_group     object
 6   drug_class     object
dtypes: int64(2), object(5)
memory usage: 160.5+ MB


# 3. Summarizing member health status based on latest diagnosis

I decided to go with the latest diagnosis for each patient since that is the true health status at that point of time for the member and it would encompass any details from previous visits. Hence, my assumption is that all previous visits lead up to the final diagnosis and that there are no separate diagnoses.  

### 3.1 Changing the diagnosis variable in the claim lines dataset to match the corresponding name in the CCS dataset so that we can jon the two datasets on that variable

In [10]:
#Renaming diagnosis column
df_claims = df_claims.rename(columns={'diag1':'diag'})
df_claims.head()

,record_id,member_id,date_svc,diag
0,57738,M0000001,2015-12-06,N92.6
1,57750,M0000001,2015-12-06,O26.842
2,65072,M0000001,2015-12-13,O26.842
3,201796,M0000001,2016-02-29,O26.843
4,267197,M0000001,2016-03-27,O26.843


### 3.2 Re-formatting diagnosis codes in claim lines dataset to remove the periods so that they match the corresponding diagnosis codes in the CCS dataset.

In [11]:
#Removing any '.' characters in the diagnosis codes 
df_claims['diag'] = df_claims['diag'].str.replace('.','')
df_claims.head(10)

,record_id,member_id,date_svc,diag
0,57738,M0000001,2015-12-06,N926
1,57750,M0000001,2015-12-06,O26842
2,65072,M0000001,2015-12-13,O26842
3,201796,M0000001,2016-02-29,O26843
4,267197,M0000001,2016-03-27,O26843
5,284553,M0000001,2016-04-03,O26843
6,301751,M0000001,2016-04-10,O26843
7,304120,M0000001,2016-04-11,O26843
8,309100,M0000001,2016-04-13,O3421
9,309112,M0000001,2016-04-13,O80


The modified dataframe above shows that the formatting worked correctly and we can proceed to the next step. 

### 3.3 Merging the claim lines and CCS datasets to map the member IDs to the diagnosis descriptions

In [12]:
#Joining the claim_lines and CSS datasets
df_diag = df_claims.merge(df_ccs,how='inner',on='diag')
df_diag.head(10)

,record_id,member_id,date_svc,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
0,57738,M0000001,2015-12-06,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
1,1563758,M0000131,2018-01-20,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
2,125770,M0000194,2016-01-24,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
3,652693,M0000231,2016-09-04,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
4,1721525,M0000252,2018-03-01,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
5,1308475,M0000340,2017-09-07,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
6,169753,M0000404,2016-02-15,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
7,1501163,M0000407,2017-12-30,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
8,578127,M0000495,2016-08-03,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
9,543887,M0001931,2016-07-19,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders


### 3.4 Finding the most recent diagnosis for each member

In [13]:
#Converting the dates from string to Pandas datetime format for easier manipulation
df_diag['date_svc'] = df_diag['date_svc'].apply(pd.to_datetime)
df_diag.head()

,record_id,member_id,date_svc,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
0,57738,M0000001,2015-12-06,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
1,1563758,M0000131,2018-01-20,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
2,125770,M0000194,2016-01-24,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
3,652693,M0000231,2016-09-04,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders
4,1721525,M0000252,2018-03-01,N926,"Irregular menstruation, unspecified",Diseases of the genitourinary system,Diseases of female genital organs,Menstrual disorders


In [15]:
#Grouping by member ID and finding the latest member diagnosis
df_latest_date = df_diag.loc[df_diag.groupby('member_id')['date_svc'].idxmax()]
df_latest_date.head()

,record_id,member_id,date_svc,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
5012,319629,M0000001,2016-04-17,Z3801,"Single liveborn infant, delivered by cesarean",Certain conditions originating in the perinata...,Liveborn [218.],Liveborn
98015,1170773,M0000002,2017-06-05,I25118,Athscl heart disease of native cor art w oth a...,Diseases of the circulatory system,Diseases of the heart,Coronary atherosclerosis and other heart disease
98335,1275838,M0000003,2017-08-16,M2022,"Hallux rigidus, left foot",Diseases of the musculoskeletal system and con...,Acquired deformities,Acquired foot deformities
98504,1793014,M0000004,2018-03-19,Z111,Encounter for screening for respiratory tuberc...,Infectious and parasitic diseases,Immunizations and screening for infectious dis...,Immunizations and screening for infectious dis...
101978,1658297,M0000005,2018-02-14,G43001,"Migraine w/o aura, not intractable, with statu...",Diseases of the nervous system and sense organs,Headache; including migraine [84.],Headache; including migraine


In [21]:
print ("Number of unique categories for diag_desc: ",len(df_latest_date['diag_desc'].value_counts()))

Number of unique categories for diag_desc:  9418


In [22]:
print ("Number of unique categories for ccs_1_desc: ",len(df_latest_date['ccs_1_desc'].value_counts()))

Number of unique categories for ccs_1_desc:  18


In [25]:
print ("Number of unique categories for ccs_2_desc: ",len(df_latest_date['ccs_2_desc'].value_counts()))

Number of unique categories for ccs_2_desc:  134


In [24]:
print ("Number of unique categories for ccs_3_desc: ",len(df_latest_date['ccs_3_desc'].value_counts()))

Number of unique categories for ccs_3_desc:  267


From the analysis of the unique categories for each diagnosis description, it seems that **ccs_1_desc** seems to have the least number of unique categories which means it's the highest level of diagnosis available, followed by **ccs_2_desc** and **ccs_3_desc**. Lastly, **diag_desc** provides the lowest level description and hence contains the most unique values.

Even though **diag_desc** provides a good amount of detail for the diagnosis description, I have chosen to go with **ccs_3_desc** since it has fewer categories and still provides a good amount information regarding the diagnosis. Since it only has 267 categories compared to **diag_desc**, it will also quicken the modeling process. The resulting description is stored in a new variable called **health_status**.

### 3.5 Compiling the diagnosis descriptions for each unique member

In [26]:
df_latest_date['health_status'] = df_latest_date.apply((lambda x: 'Member ' + x[1] + ' was diagnosed with ' + x[7]),axis=1)
df_latest_date['health_status'] = df_latest_date['health_status'].apply(str.lower)
df_latest_date['health_status'] = df_latest_date['health_status'].apply(lambda x: x[0].upper() + x[1:])
df_latest_date['health_status'] = df_latest_date['health_status'].apply(lambda x: x[:7] + x[7].upper() + x[8:])
df_latest_date.head()

,record_id,member_id,date_svc,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc,health_status
5012,319629,M0000001,2016-04-17,Z3801,"Single liveborn infant, delivered by cesarean",Certain conditions originating in the perinata...,Liveborn [218.],Liveborn,Member M0000001 was diagnosed with liveborn
98015,1170773,M0000002,2017-06-05,I25118,Athscl heart disease of native cor art w oth a...,Diseases of the circulatory system,Diseases of the heart,Coronary atherosclerosis and other heart disease,Member M0000002 was diagnosed with coronary at...
98335,1275838,M0000003,2017-08-16,M2022,"Hallux rigidus, left foot",Diseases of the musculoskeletal system and con...,Acquired deformities,Acquired foot deformities,Member M0000003 was diagnosed with acquired fo...
98504,1793014,M0000004,2018-03-19,Z111,Encounter for screening for respiratory tuberc...,Infectious and parasitic diseases,Immunizations and screening for infectious dis...,Immunizations and screening for infectious dis...,Member M0000004 was diagnosed with immunizatio...
101978,1658297,M0000005,2018-02-14,G43001,"Migraine w/o aura, not intractable, with statu...",Diseases of the nervous system and sense organs,Headache; including migraine [84.],Headache; including migraine,Member M0000005 was diagnosed with headache; i...


In [27]:
df_health_status = df_latest_date[['member_id','health_status']]
df_health_status.index = range(1,df_health_status.shape[0]+1)
df_health_status.head()

,member_id,health_status
1,M0000001,Member M0000001 was diagnosed with liveborn
2,M0000002,Member M0000002 was diagnosed with coronary at...
3,M0000003,Member M0000003 was diagnosed with acquired fo...
4,M0000004,Member M0000004 was diagnosed with immunizatio...
5,M0000005,Member M0000005 was diagnosed with headache; i...


### 3.6 Exporting the health statuses as a CSV file

In [28]:
df_health_status.to_csv('member_health_statuses.csv')

### 3.7 Previewing a few of the health statuses

In [29]:
df_health_status.iloc[0,-1]

'Member M0000001 was diagnosed with liveborn'

In [30]:
df_health_status.iloc[1,-1]

'Member M0000002 was diagnosed with coronary atherosclerosis and other heart disease'

In [31]:
df_health_status.iloc[2,-1]

'Member M0000003 was diagnosed with acquired foot deformities'

# 4. Predicting diagnosis based on prescription

For this part, we can use the prescription data as independent variables and try to predict the target variable which is the health status. Hence, this would be a multi-class classification problem. We can use the accuracy, ROC AUC, precision and recall metrics to evaluate the model. 

The idea is to see if all the prescribed medications for members (not only the latest) can narrow down the health status with the goal of pinpointing the ultimate (latest) health condition early on. 

Another way the prediction problem could have been setup is by using **diag_desc** as the target variable since it provides more detail, however it contains over 9000 categories which would make the training process significantly slower. We use **ccs_3_desc** since it does not contain as many categories but provides enough detail. 

First we would need to merge the health status data with the prescription data so that we have that mapping. Then we could just use the prescriptions and health statuses to perform the modeling. 

In [36]:
print ("Number of unique categories for drug_category: ",len(df_drugs['drug_category'].value_counts()))

Number of unique categories for drug_category:  92


In [37]:
print ("Number of unique categories for drug_group: ",len(df_drugs['drug_group'].value_counts()))

Number of unique categories for drug_group:  464


In [38]:
print ("Number of unique categories for drug_class: ",len(df_drugs['drug_class'].value_counts()))

Number of unique categories for drug_class:  687


In [39]:
df_drugs.shape

(3005934, 7)

### 4.1 Merging health status and prescription datasets

In [30]:
df_model = df_drugs.merge(df_health_status,how='inner',on='member_id')
df_model.head()

,record_id,member_id,date_svc,ndc,drug_category,drug_group,drug_class,health_status
0,4115084976453758912,M0023556,2016-05-08,51285040702,Estrogens,Estrogens,Estrogens,Member M0023556 was diagnosed with diabetes me...
1,633634268929772520,M0023556,2016-11-15,46110181,Estrogens,Estrogens,Estrogens,Member M0023556 was diagnosed with diabetes me...
2,8828344746674933500,M0023556,2018-01-10,99073012101,Diagnostic Products,Diagnostic Tests,Diagnostic Tests,Member M0023556 was diagnosed with diabetes me...
3,175380569212871566,M0023556,2017-07-20,46110181,Estrogens,Estrogens,Estrogens,Member M0023556 was diagnosed with diabetes me...
4,4111617532992638126,M0023556,2017-05-27,46110181,Estrogens,Estrogens,Estrogens,Member M0023556 was diagnosed with diabetes me...


In [31]:
df_model.shape

(2749752, 8)

### 4.2 Removing variables not useful for prediction

Since we only need need the prescription information in order to predict the health status classes, we can remove the following variables which provide little information for the modeling process: 
- record_id
- member_id
- date_svc: Provides same information as **drug_group** and **drug_class**
- ndc
- drug_group: This variable was removed since it provided a lot of the same information as **drug_class**. During prediction, if we only have this information, we can easily map it from drug_class. 

In [32]:
df_model = df_model.drop(['record_id','member_id','date_svc','ndc','drug_group'],axis=1)
df_model.head()

,drug_category,drug_class,health_status
0,Estrogens,Estrogens,Member M0023556 was diagnosed with diabetes me...
1,Estrogens,Estrogens,Member M0023556 was diagnosed with diabetes me...
2,Diagnostic Products,Diagnostic Tests,Member M0023556 was diagnosed with diabetes me...
3,Estrogens,Estrogens,Member M0023556 was diagnosed with diabetes me...
4,Estrogens,Estrogens,Member M0023556 was diagnosed with diabetes me...


### 4.3 Sampling part of the dataset due to memory constraints

In [33]:
df_model = df_model.sample(frac=0.3,random_state=0)

In [34]:
df_model.shape

(274975, 3)

### 4.4 Pre-processing and separating dataset into X and Y variables

In [35]:
#Label encoding categorical variables
le = LabelEncoder()

le.fit_transform(df_model['drug_category'])
df_model["drug_category"] = le.transform(df_model["drug_category"])

le.fit_transform(df_model['drug_class'])
df_model["drug_class"] = le.transform(df_model["drug_class"])

le.fit_transform(df_model['health_status'])
df_model["health_status"] = le.transform(df_model["health_status"])

In [36]:
X = df_model.drop('health_status',axis=1)
Y = df_model['health_status']
print (X.shape)
print (Y.shape)

(274975, 2)
(274975,)


In [37]:
X.head()

,drug_category,drug_class
1921782,23,280
2730628,23,280
1383841,16,488
2224594,15,67
221136,46,330


### 4.5 Separating the data further into train and test sets

Normally, it would also be useful to have a validation set (third set) to test the final model but we will stick with just the train and test sets for this example. 

In [38]:
#Train/Test Split
x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=0,test_size=0.3)
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(192482, 2)
(82493, 2)
(192482,)
(82493,)


### 4.6 Modeling and evaluation

In [2]:
def model_predict(model,x_train,x_test,y_train,y_test):  

  x_train = pd.DataFrame(x_train)
  x_train.columns = X.columns
  x_test = pd.DataFrame(x_test)
  x_test.columns = X.columns
  y_train = pd.DataFrame(y_train)
  y_test = pd.DataFrame(y_test)

  #Modeling and prediction
  model = model.fit(x_train,y_train)
  prediction = model.predict(x_test)

  #Calculating prediction probabilities
  lr_probs = model.predict_proba(x_test)
  lr_probs_pos = lr_probs[:,1]
  lr_probs_neg = lr_probs[:,0]

  #lr_auc_pos = roc_auc_score(y_test, lr_probs_pos, multi_class='ovr')
  #tn, fp, fn, tp = confusion_matrix(y_test.values, prediction).ravel()

  #Calculating scoring metrics
  precision = precision_score(y_test.values,prediction,average=None)
  accuracy = accuracy_score(y_test.values,prediction)
  #roc_auc = lr_auc_pos
  recall = recall_score(y_test.values,prediction,average=None)
  f1 = f1_score(y_test.values,prediction,average=None)
  #specificity = tn/(tn+fp)

  #print ("ROC AUC: ",roc_auc)
  print ("Accuracy: ",accuracy)
  print ("Precision: ",precision)
  print ("Recall: ",recall)
  print ("F1 score: ",f1)
  #print ("Specificity: ",specificity)

  '''
  #Calculating ROC AUC values
  lr_auc = roc_auc_score(y_test, prediction, multi_class='ovr')
  lr_auc_pos = roc_auc_score(y_test, lr_probs_pos, multi_class='ovr')
  lr_auc_neg = roc_auc_score(y_test, lr_probs_neg, multi_class='ovr')

  print('Combined ROC AUC=%.3f' % (lr_auc))
  print('Severe class ROC AUC=%.3f' % (lr_auc_pos))
  print('Not severe class ROC AUC=%.3f' % (lr_auc_neg))

  lr_fpr, lr_tpr, thresh = roc_curve(y_test, prediction)
  lr_fpr_neg, lr_tpr_neg, thresh = roc_curve(y_test, lr_probs[:,0])
  lr_fpr_pos, lr_tpr_pos, thresh = roc_curve(y_test, lr_probs_pos)

  tn, fp, fn, tp = confusion_matrix(y_test.values, prediction).ravel()
  '''

In [ ]:
model_predict(LGBMClassifier(objective='multiclass',random_state=0),x_train,x_test,y_train,y_test)

         health_status
1247950          57834
1074429          39421
177556           93125
146291           41828
724080           44036
...                ...
881494           17912
1021883          47250
1065396          48808
100646           32202
1547644          60234

[82493 rows x 1 columns]
(82493, 1)


Trying to run the model with the given data tends to crash the session as it exceeds the amount of RAM available.

Possible options for preventing crashes: 
- Use a smaller sample size for the training data
- Reduce the number of categories in variables which would decrease the training time but make the categories more general
- Use **ccs_1_desc** as the health status since it only contains 18 categories

I ran out of time to fix these issues. Above, I tried to run the model in a 'one vs all' manner for a multi-class classification problem for predicting the various health statuses. I used the LightGBM classifier since it tends to be quicker and more accurate than the other tree-based models. Also, it requires very less preprocessing for the training data. 